In [1]:
import json
import random

In [2]:
traits = ["o", "c", "e", "a", "n"]
predict_dict = {}
for t in traits:
    predict_dict[t] = {}
    with open(f"/data/user_data/wenkail/llm_personality/generator/data/outputs/generated_predictions_{t}.jsonl", "r") as f:
        store_low, store_high = [], []
        for i in f.readlines():
            tmp = json.loads(i)
            if " - low\n" in tmp['prompt']:
                store_low.append(tmp)
            elif " - high\n" in tmp['prompt']:
                store_high.append(tmp)
        predict_dict[t]['low'] = store_low
        predict_dict[t]['high'] = store_high

In [3]:
sub_predict = {}
for t in traits:
    sub_predict[t] = {}
    for l in ["low", "high"]:
        sample_size = min(len(predict_dict[t][l]), 100)
        random_sample = random.sample(predict_dict[t][l], sample_size)
        sub_predict[t][l] = random_sample

In [4]:
sub_predict.keys(), sub_predict['o'].keys(), len(sub_predict['o']['high'])

(dict_keys(['o', 'c', 'e', 'a', 'n']), dict_keys(['low', 'high']), 100)

In [5]:
sub_predict['o']['high'][0]

{'prompt': 'user\n\nHelp me complete the sentence with certain Big Five Personality: Openness - high\nidgaf who come to myassistant\n\n',
 'label': 'party truthfully. i just want it to be live asfuck.',
 'predict': "house i'm not going to the party"}

In [6]:
import re

def extract_starting_sentence(text):
    # Regex pattern to capture text before "assistant\n\n" while accounting for "- high\n" or "- low\n"
    match = re.search(r'- (high|low)\n(.+?)(?=assistant\n\n)', text, re.DOTALL)
    if match:
        starting_sentence = match.group(2).strip()
        return starting_sentence
    return None

# Example input
input_text = "user\n\nHelp me complete the sentence with certain Big Five Personality: Openness - high\nhmm... i'm starting to hateassistant\n\n"

# Extracting the starting sentence
starting_sentence = extract_starting_sentence(input_text)
print(starting_sentence)


hmm... i'm starting to hate


In [7]:
for t in traits:
    for l in ["low", "high"]:
        for i in range(100):
            sub_predict[t][l][i]['psychgenerator_data'] = extract_starting_sentence(sub_predict[t][l][i]['prompt']) + " " + sub_predict[t][l][i]['label']
            sub_predict[t][l][i]['expert_generator_data'] = extract_starting_sentence(sub_predict[t][l][i]['prompt']) + " " + sub_predict[t][l][i]['predict']

In [8]:
sub_predict['o']['high'][0]

{'prompt': 'user\n\nHelp me complete the sentence with certain Big Five Personality: Openness - high\nidgaf who come to myassistant\n\n',
 'label': 'party truthfully. i just want it to be live asfuck.',
 'predict': "house i'm not going to the party",
 'psychgenerator_data': 'idgaf who come to my party truthfully. i just want it to be live asfuck.',
 'expert_generator_data': "idgaf who come to my house i'm not going to the party"}

In [9]:
out_f = open("baseline_source_data.json", 'w')
json.dump(sub_predict, out_f, indent=2, ensure_ascii=False)
out_f.flush()